# Retrieving news from Event Registry

In [1]:
%matplotlib inline
from eventregistry import *
from eventRegistryHandler import *
from dataHandler import *
import seaborn as sns
import os

er = EventRegistry(apiKey = "1f418876-731f-4041-b042-2d6d26d93fc7")
#"9eb5d263-141a-4450-a8e3-5d3c20158798"
#ricardooliveira@copin.ufcg.edu.br
#linha743

using user provided API key for making requests
Event Registry host: http://eventregistry.org
Text analytics host: http://analytics.eventregistry.org


In [2]:
def get_topic(word):
    for key, values in topics.items():
        if word in values:
            return key
    return None


def file_exists(file_name):
    return os.path.isfile(file_name)

In [3]:
bias_file = pd.read_csv("MediaBiasFactCheck/outlets_political_stance")
columns = ['categories','uri','url','source.uri','source.title','title','body','date','dataType','lang','source.location.country.label.eng','source.location.label.eng','sentiment','sim']

FINAL_loc = "title"
FINAL_sim = 0.9
FINAL_n = -1

In [4]:
keywords = {"por":["Venezuela","Síria",["Lava","Jato"],"Moro"],"deu":["Venezuela","Syrien","Moro"],"eng":["Venezuela","Syria","Moro"]}
topics = {"venezuela":["Venezuela"], "syria":["Síria","Syrien","Syria"],"lavajato":["Lava_Jato"],"sergiomoro":["Moro"]}

In [6]:
for lang in keywords.keys():
    for key in keywords[lang]:
        print key, lang
        
        topic_name = key
        if type(key) == list:
            topic_name = "_".join(key)
            
        file_name = "EventRegistryData/backup/news_"+topic_name+"_"+lang+".csv"
        exists = file_exists(file_name)
        
        data = None
        last_update = None
        if exists:
            data = pd.read_csv(file_name)
            last_update = data.date.max()
        
        last_update = "2019-06-09"
        FINAL_n = 30
        lang_iter = get_news_iterator(er=er, keywords=key, language=lang, loc=FINAL_loc, n_items=FINAL_n, date_start = last_update, outlets_ids=None)
        new_data = retrieve_news_2(lang_iter, columns)
        new_data = add_country_location(new_data)
        new_data = add_bias_information(bias=bias_file, news=new_data, match_similarity=FINAL_sim)
        
        update_data = new_data.append(data)
        update_data["topic"] = get_topic(topic_name)
        update_data.to_csv(file_name, index = False, header=True, encoding="utf-8")
        

Venezuela por
Síria por
['Lava', 'Jato'] por
Moro por
Venezuela deu
Syrien deu
Moro deu
Venezuela eng
Syria eng
Moro eng


In [ ]:
#deu_outlets_ids =["jungefreiheit.de"]#,"faz.net","spiegel.de","taz.de","zeit.de","bild.de","welt.de"
#pt_outlets_ids =["brasil247.com","cartacapital.com.br","oantagonista.com","veja.abril.com.br"]
#eng_outlets_ids =["huffpost.com"],"foxnews.com","breitbart.com","washingtonpost.com","reuters.com"]

## Dataset Description

### Joining tables

In [ ]:
import glob

path = r'EventRegistryData/backup/'
all_files = glob.glob(path + "/*.csv")
li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None)
    li.append(df)

news = pd.concat(li, axis=0, ignore_index=True)
news["subject"] = news.title.apply(lambda x: "venezuela" if "venezuela" in x.lower() else "syria")

### Dataset Overview

In [ ]:
print "min: "+news.date.min()
print "max: "+news.date.max()
print len(news.date.unique())

In [ ]:
news.subject.value_counts().plot(kind="bar")

In [ ]:
news.lang.value_counts().plot(kind="bar")

In [ ]:
len(news["country_location"].unique())

In [ ]:
news.country_location.value_counts()[0:10].plot(use_index=True, kind="bar")

In [ ]:
len(news["source.title"].unique())

In [ ]:
news["source.title"].value_counts()[0:10].plot(kind="bar")

### USA, UK, Canada, Germany and Brazil

In [ ]:
isin = news.country_location.isin(["United States","Brazil","Germany","United Kingdom","Canada"])
news_countries = news[isin]

In [ ]:
news_countries.shape

Number of news by country:

In [ ]:
news_countries.country_location.value_counts().plot(kind="bar")

### Do the news subjectivities rates changes according to the countries news media?

Are the news' subjectivities, over a predefined subject, of Brazilian, Germany and American news outlets significantly different?

Number of news by country:

In [ ]:
sns.catplot(x="subject", hue="country_location", kind="count", palette="pastel", edgecolor=".6", data=news_countries)

Number of news by language and country:

In [ ]:
sns.catplot(x="country_location", hue="lang", col="subject", kind="count", palette="pastel", edgecolor=".6", data=news_countries)

### Does the news outlets political slant have an influence on the subjectivity rates?

News by political orientation in all countries:

In [ ]:
news.stance.value_counts().plot(kind="bar")

News by political orientation in USA, UK, Canada, Germany and Brazil:

In [ ]:
news_countries.stance.value_counts().plot(kind="bar")

Political slant by country and subject:

In [ ]:
sns.catplot(x="country_location", hue="stance", col="subject", kind="count", palette="pastel", edgecolor=".6", data=news_countries)

Encontrar um contexto em que a cobertura da guerra da siria nao seja significantemente menor que a crise da venezuela